In [79]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import requests as rq
from urllib.parse import urlencode

from datetime import timedelta

%matplotlib inline

In [2]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
customer_data = 'https://disk.yandex.ru/d/Ta-weYfGM5CHjg'

# Получаем загрузочную ссылку
download_url_customer_data = rq.get(base_url + urlencode(dict(public_key=customer_data))).json()['href']
download_response = rq.get(download_url_customer_data)

In [3]:
customers_df = pd.read_csv(download_url_customer_data, parse_dates=['date'])

### Описание данных 

- **`date`** – дата совершения события

- **`event`** - событие
    + *app_install – установка приложения*
    + *app_start – открыть приложения*
    + *registration – зарегистрироваться*
    + *search – перейти на страницу поиска товаров (каталог)*
    + *open_item – открыть товар*
    + *choose_item – отправить товар в корзину*
    + *tap_basket – перейти в корзину*
    + *purchase – подтверждение покупки*
    
- **`purchase_sum`** – стоимость покупки (при совершении события ‘purchase’)

- **`os_name`** – платформа пользователя

- **`device_id`** – идентификатор устройства пользователя

- **`gender`** – пол пользователя

- **`city`** – город пользователя

- **`urm_source`** – канал, с которого пришел пользователь
    + *yandex-direct – Яндекс директ*
    + *google_ads – реклама в Google*
    + *vk_ads – реклама в ВК*
    + *instagram_ads – реклама в instagram*
    + *facebook_ads – реклама в facebook*
    + *referral – акция «приведи друга»*
    + ***Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы***  

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [4]:
customers_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [5]:
customers_df.shape

(2747968, 8)

In [6]:
customers_df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

In [7]:
customers_df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [8]:
f'date_time from {customers_df.date.min()} to {customers_df.date.max()}'

'date_time from 2020-01-01 00:00:00 to 2020-03-31 00:00:00'

In [9]:
customers_df.os_name.unique()

array(['android', 'ios'], dtype=object)

In [10]:
customers_df.event.value_counts().sort_values(ascending=False)

app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: event, dtype: int64

In [11]:
customers_df.city.value_counts().sort_values(ascending=False)

Moscow              1491481
Saint-Petersburg    1256487
Name: city, dtype: int64

In [12]:
customers_df.utm_source.value_counts().sort_values(ascending=False)

-                610458
vk_ads           418456
yandex-direct    390232
instagram_ads    382891
google_ads       374257
facebook_ads     336953
referal          234721
Name: utm_source, dtype: int64

In [13]:
f'number of rows:{customers_df.device_id.count()}, and number of unique device_ids(users):{customers_df.device_id.nunique()}'

'number of rows:2747968, and number of unique device_ids(users):190884'

In [14]:
f'avg num of actions per customer:{customers_df.device_id.count()/customers_df.device_id.nunique()}'

'avg num of actions per customer:14.396010142285368'

### Задание 4.
MAU февраля:

In [15]:
customers_df.loc[customers_df.date.dt.month == 2].device_id.nunique()

75032

### Задание 5.
Количество установок в январе:



In [16]:
customers_df.loc[(customers_df.date.dt.month == 1) & (customers_df.event == 'app_install')].event.count()

80297

### Задание 6.
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [113]:
install_cohorts = customers_df.query('event == "app_install"')

In [116]:
install_cohorts.shape

(154597, 8)

In [114]:
install_cohorts.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal


In [117]:
first_purchase = customers_df \
    .query('event == "purchase"')\
    .rename(columns={'date':'first_purchase_date'}) \
    .groupby('device_id', as_index=False) \
    .first_purchase_date.min()

In [119]:
first_purchase.head()

,device_id,first_purchase_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07


In [121]:
df_cohort_first_purch = install_cohorts.merge(first_purchase, on=['device_id'])

In [123]:
df_cohort_first_purch.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_purchase_date
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10
1,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01
2,2020-01-01,app_install,NaN,android,8423608,female,Saint-Petersburg,referal,2020-01-06
3,2020-01-01,app_install,NaN,android,532710,male,Moscow,yandex-direct,2020-01-03
4,2020-01-01,app_install,NaN,android,528071,female,Saint-Petersburg,vk_ads,2020-01-02


In [126]:
df_cohort_first_purch['day_diff'] = df_cohort_first_purch.first_purchase_date - df_cohort_first_purch.date

In [150]:
df_cohort_first_purch.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,first_purchase_date,day_diff
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10,9 days
1,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01,0 days
2,2020-01-01,app_install,NaN,android,8423608,female,Saint-Petersburg,referal,2020-01-06,5 days
3,2020-01-01,app_install,NaN,android,532710,male,Moscow,yandex-direct,2020-01-03,2 days
4,2020-01-01,app_install,NaN,android,528071,female,Saint-Petersburg,vk_ads,2020-01-02,1 days


In [131]:
app_install = install_cohorts.groupby('date', as_index=False) \
    .agg({'device_id':'count'}).rename(columns={'device_id':'app_install'})

In [133]:
app_install.shape

(91, 2)

In [154]:
first_purchase = df_cohort_first_purch \
    .loc[df_cohort_first_purch.first_purchase_date - df_cohort_first_purch.date <= '7 days'] \
    .groupby('date', as_index=False) \
    .agg({'device_id':'count'}).rename(columns={'device_id':'first_purchase'})

In [155]:
first_purchase.shape

(91, 2)

In [156]:
final_df = first_purchase.merge(app_install,on=['date'])

In [157]:
final_df['CR'] = round(final_df.first_purchase / final_df.app_install*100, 1)

In [158]:
final_df.sort_values('CR', ascending=False)

,date,first_purchase,app_install,CR
0,2020-01-01,1408,3579,39.3
8,2020-01-09,558,1424,39.2
14,2020-01-15,1650,4310,38.3
13,2020-01-14,1973,5173,38.1
1,2020-01-02,1186,3144,37.7
...,...,...,...,...
77,2020-03-18,235,1171,20.1
88,2020-03-29,223,1117,20.0
81,2020-03-22,251,1261,19.9
87,2020-03-28,209,1091,19.2


### Задание 7.
Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой). Формат числа - с точкой. Пример: 66.4.

In [173]:
final_df.head(1).CR

0    39.3
Name: CR, dtype: float64

### Задание 8.

С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.

In [172]:
customers_df.query('event == "app_install"').utm_source.value_counts().head(2)

-                32460
yandex-direct    29368
Name: utm_source, dtype: int64

### Задание 9.

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [182]:
customers_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [189]:
reg_dates = customers_df.query('event == "register"')[['date', 'device_id']].rename(columns={'date':'reg_date'})

In [190]:
cutomers_reg_df = customers_df.merge(reg_dates, on='device_id', how='left')

In [191]:
cutomers_reg_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,reg_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT


In [193]:
cutomers_reg_df.shape

(2747968, 9)

In [192]:
cutomers_reg_df.reg_date.isna().sum()

542002

In [194]:
cutomers_reg_df.reg_date.notna().sum()

2205966

In [197]:
no_reg_df = cutomers_reg_df.loc[cutomers_reg_df.date < cutomers_reg_df.reg_date]

In [198]:
yes_reg_df = cutomers_reg_df.loc[cutomers_reg_df.date >= cutomers_reg_df.reg_date]

In [199]:
yes_reg_df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,reg_date
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-,2020-01-01
10,2020-01-01,app_start,NaN,android,11876266,female,Moscow,yandex-direct,2020-01-01
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-31


  + *app_install – установка приложения*
  + *app_start – открыть приложения*
  + *registration – зарегистрироваться*
  + *search – перейти на страницу поиска товаров (каталог)*
  + *open_item – открыть товар*
  + *choose_item – отправить товар в корзину*
  + *tap_basket – перейти в корзину*
  + *purchase – подтверждение покупки*

In [204]:
cr_df = yes_reg_df.groupby('event', as_index=False).agg({'device_id':'count'})

In [205]:
cr_df

,event,device_id
0,app_install,42448
1,app_start,519754
2,choose_item,392783
3,purchase,141383
4,register,78310
5,search,495411
6,tap_basket,295566


In [218]:
cr_df = cr_df.reindex(index = [0, 1, 4, 5, 2, 6, 3])

In [219]:
cr_df['result1'] = cr_df['device_id'].div(cr_df['device_id'].shift()) * 100

In [220]:
cr_df

,event,device_id,result1
0,app_install,42448,NaN
1,app_start,519754,1224.448737
4,register,78310,15.066743
5,search,495411,632.628017
2,choose_item,392783,79.284271
6,tap_basket,295566,75.249183
3,purchase,141383,47.834663


### Задание 10.
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [221]:
customers_df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [233]:
app_start = customers_df \
    .query('event == "app_start"') \
    .groupby('utm_source', as_index=False) \
    .device_id.nunique().rename(columns={'device_id':'num_start'})

In [236]:
purchase = customers_df \
    .query('event == "purchase"') \
    .groupby('utm_source', as_index=False) \
    .device_id.nunique().rename(columns={'device_id':'purchases'})

In [238]:
final_df = app_start.merge(purchase, on='utm_source')

In [242]:
final_df['cr'] = final_df.purchases / final_df.num_start * 100

In [244]:
final_df.sort_values('cr')

,utm_source,num_start,purchases,cr
6,yandex-direct,40707,12028,29.547744
2,google_ads,38092,11339,29.767405
0,-,52268,16598,31.755567
3,instagram_ads,31045,10762,34.665808
1,facebook_ads,25957,9017,34.738221
5,vk_ads,34079,12364,36.280407
4,referal,15924,6362,39.952273


### Задание 11.

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [255]:
customers_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [254]:
customers_df \
    .query('event == "purchase"') \
    .groupby(['device_id', 'utm_source'], as_index=False) \
    .agg({'date':'min', 'purchase_sum':'sum'}) \
    .groupby('utm_source', as_index=False) \
    .purchase_sum.median().sort_values('purchase_sum', ascending=False).head(1)

,utm_source,purchase_sum
4,referal,530.0


### Задание 12.

Данные по затратам на рекламу:

- Яндекс – 10 491 707 руб.

- Гугл – 10 534 878 руб.

- Фейсбук – 8 590 498 руб.

- Инстаграм – 8 561626 руб.

- ВК – 9 553 531руб.

Какой платный канал привлечения имеет самый высокий ROMI?
**ROMI** = (Валовая прибыль − Маркетинговые расходы) / Маркетинговые расходы

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

In [318]:
revenue_df = customers_df.groupby('utm_source', as_index=False) \
    .purchase_sum.sum()

In [319]:
marketing_cost = {'yandex-direct':10491707, 
                   'google_ads':10534878, 
                   'facebook_ads':8590498,
                   'instagram_ads':8561626, 
                   'vk_ads':9553531}

In [320]:
marketing_cost = pd.Series(marketing_cost, name='cost') \
    .to_frame().reset_index().rename(columns={'index':'utm_source'})

In [321]:
marketing_cost

,utm_source,cost
0,yandex-direct,10491707
1,google_ads,10534878
2,facebook_ads,8590498
3,instagram_ads,8561626
4,vk_ads,9553531


In [322]:
revenue_df = revenue_df.merge(marketing_cost, on='utm_source')

In [323]:
revenue_df

,utm_source,purchase_sum,cost
0,facebook_ads,12249901.0,8590498
1,google_ads,12868276.0,10534878
2,instagram_ads,14546969.0,8561626
3,vk_ads,16389652.5,9553531
4,yandex-direct,13915368.0,10491707


In [324]:
revenue_df['ROMI'] = (revenue_df.purchase_sum - revenue_df.cost)/revenue_df.cost

In [325]:
revenue_df.sort_values('ROMI').tail(1)

,utm_source,purchase_sum,cost,ROMI
3,vk_ads,16389652.5,9553531,0.71556
